# **seq2seq 기계 번역**

## **1. 데이터 전처리**

In [1]:
import os
import shutil
import zipfile
import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

fra-eng.zip data download link : http://www.manythings.org/anki

In [6]:
zf = zipfile.ZipFile('fra-eng.zip')
zf.extractall() 
zf.close()
# with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
#     zip_ref.extractall(path)

source sentence - target sentence - license 로 이뤄진 데이터셋에서 라이센스 열 지워주기

In [19]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

190206

전체 샘플 수 190206개

In [20]:
lines = lines.loc[:, 'src':'tar']   # 모든 행에 대해 'src', 'tar' 열 데이터만 가져와라
lines = lines[0:60000]  #6만개의 샘플만 사용해 기계 번역기 구축
lines.sample(10)

,src,tar
14608,It was unlocked.,Elle était déverrouillée.
26371,We were prisoners.,Nous étions prisonniers.
50422,Tell me what to think.,Dis-moi que penser !
14563,It looked funny.,Ça avait l'air drôle.
13880,I love your son.,Je suis amoureuse de ton fils.
10306,I love secrets.,J'adore les secrets.
33551,Did you do all this?,As-tu fait tout ceci ?
19224,It's the big one.,C'est le gros.
46564,Do you know this word?,Connais-tu ce mot ?
12144,We were joking.,Nous étions en train de plaisanter.


랜덤 선택된 10개의 샘플들  
번역 문장에 해당하는 프랑스어 데이터엔 시작과 종료를 의미하는 sos, eos 심볼을 넣어주어야 함.  
여기선 sos, eos 대신 '\t', '\n'을 사용

In [21]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
44433,Tom is such a coward.,\t Tom n'est qu'un lâche. \n
4554,Have a drink.,\t Prenez une boisson ! \n
33848,Everybody loves him.,\t Tout le monde l'aime. \n
40496,He was almost asleep.,\t Il dormait presque. \n
48358,I made a huge mistake.,\t J'ai commis une énorme erreur. \n
40370,He is reading a book.,\t Il est en train de lire un livre. \n
22401,Everyone is tired.,\t Tout le monde est fatigué. \n
44046,They're not all busy.,\t Ils ne sont pas tous occupés. \n
57440,The cat caught a mouse.,\t Le chat a attrapé la souris. \n
46461,Did you talk about me?,\t As-tu parlé de moi ? \n


이제 글자 집합을 생성해야 함.  
단어 집합이 아니라 글자 집합이라 하는 이유는 토큰 단위가 단어가 아니라 글자이기 때문.

In [22]:
src_vocab = set()
for line in lines.src:  # 1줄씩 read
    for char in line:   # 1 글자씩 read
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [23]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print(src_vocab_size)
print(tar_vocab_size)

79
105


영어와 프랑스어에는 각각 79, 105개의 글자가 존재.  
이 중 인덱스를 임의로 부여해 일부만 출력해보자.

In [24]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


이제 각 글자에 인덱스를 부여해보자

In [25]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

이제 이를 가지고 훈련 데이터에 정수 인코딩을 수행해보자. 우선 인코더의 입력이 될 영어 문장 샘플에 대해서 정수 인코딩을 해보고, 5개의 샘플을 출력해보자.

In [26]:
encoder_input = []
for line in lines.src:
    temp_X = []
    for w in line:
        temp_X.append(src_to_index[w])
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]


디코더의 입력이 될 프랑스어 데이터에 대해서도 정수 인코딩을 해보자.

In [27]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2]]


모든 decoder_input 원소의 앞에 1, 뒤엔 2가 붙어있다. 이를 통해 sos, eos 토큰이 제대로 붙은 것을 알 수 있다.

아직 정수 인코딩을 수행해야 할 데이터가 하나 더 남아있다. 디코더의 예측값과 비교하기 위한 실제값이 필요하기 때문이다. 이 실제값에는 시작 심볼인 sos가 필요 없기 때문에 '\t'를 제거해주어야 한다.

In [28]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2]]


패딩 작업을 수행해보자. 이를 위해 영어, 프랑스어 문장 각각에서 가장 긴 샘플의 길이를 알아보자.

In [29]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

24
76


영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞추어서 패딩  
가장 긴 샘플의 길이에 맞춰서 영어 데이터의 샘플은 전부 길이가 24가 되도록 패딩하고, 프랑스어 데이터의 샘플은 전부 길이가 76이 되도록 패딩

In [30]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

이제 모든 값에 대해서 원-핫 인코딩을 수행

In [31]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

## **2. 모델 설계**

그런데 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는데 decoder_input이 왜 필요할까?

훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용한다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 만든다. 이런 상황이 반복되면 훈련 시간이 느려진다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 <i><u>실제값</u></i>을 입력으로 주는 방법을 교사 강요(Teacher forcing)라고 한다.

In [43]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [45]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

encoder_states는 디코더에 전달(컨텍스트 벡터)

In [46]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

디코더는 인코더의 마지막 은닉 상태를 초기 은닉 상태로 사용.  
출력층에 프랑스어의 단어 집합의 크기만큼 뉴런을 배치한 후 소프트맥스 함수를 사용하여 실제값과의 오차를 구한다.

In [47]:
with tf.device('/GPU:2'):
    model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/40
48000/48000 [==============================] - 23s 471us/sample - loss: 0.7348 - val_loss: 0.6605
Epoch 2/40
48000/48000 [==============================] - 15s 317us/sample - loss: 0.4538 - val_loss: 0.5307
Epoch 3/40
48000/48000 [==============================] - 15s 310us/sample - loss: 0.3828 - val_loss: 0.4723
Epoch 4/40
48000/48000 [==============================] - 14s 302us/sample - loss: 0.3414 - val_loss: 0.4395
Epoch 5/40
48000/48000 [==============================] - 15s 309us/sample - loss: 0.3130 - val_loss: 0.4088
Epoch 6/40
48000/48000 [==============================] - 15s 307us/sample - loss: 0.2919 - val_loss: 0.3925
Epoch 7/40
48000/48000 [==============================] - 15s 307us/sample - loss: 0.2758 - val_loss: 0.3820
Epoch 8/40
48000/48000 [==============================] - 15s 307us/sample - loss: 0.2628 - val_loss: 0.3713
Epoch 9/40
48000/48000 [==============================] - 15s 309us/sample - l

위에서 설정한 은닉 상태의 크기와 에포크 수는 실제로는 훈련 데이터에 과적합 상태를 불러온다.  
중간부터 검증 데이터에 대한 오차인 val_loss의 값이 올라가는데, 사실 이번 실습에서는 주어진 데이터의 양과 태스크의 특성으로 인해 훈련 과정에서 훈련 데이터의 정확도와 과적합 방지라는 두 마리 토끼를 동시에 잡기에는 쉽지 않다.  
여기서는 우선 seq2seq의 메커니즘과 짧은 문장과 긴 문장에 대한 성능 차이에 대한 확인을 중점으로 두고 훈련 데이터에 과적합 된 상태로 동작 단계로 넘어간다.

## **3. seq2seq 기계 번역기 동작시키기**

전체적인 번역 동작 단계를 정리하면 아래와 같습니다.
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
2. 상태와 SOS에 해당하는 '\t'를 디코더로 보냅니다.
3. 디코더가 EOS에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

인코더 정의

In [48]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

디코더 정의

In [49]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

인덱스로부터 단어를 만드는 index_to_src, index_to_target

In [50]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [51]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [52]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Hi.
정답 문장:  Salut ! 
번역기가 번역한 문장:  Salut. 
-----------------------------------
입력 문장: I see.
정답 문장:  Aha. 
번역기가 번역한 문장:  Je compte. 
-----------------------------------
입력 문장: Hug me.
정답 문장:  Serrez-moi dans vos bras ! 
번역기가 번역한 문장:  Serrez-moi comme un coup ! 
-----------------------------------
입력 문장: Hold it!
정답 문장:  Restez où vous êtes ! 
번역기가 번역한 문장:  Ne bougez plus ! 
-----------------------------------
입력 문장: I crashed.
정답 문장:  Je me suis écrasée. 
번역기가 번역한 문장:  Je suis détendu. 
